# Retrival Based two Agent: (Gemini) 1. REASONING AGENT + 2. RETRIEVER AGENT(RAG)

🔵 Gemini — MODEL_NAME = "models/gemini-flash-latest"

Retriever Agent: local embeddings + vector DB (Chroma).

Reasoning Agent: Gemini Flash (cloud).

Very fast and low-cost for RAG demos.

Manual agent orchestration (no native AutoGen).

Best for rapid prototyping and research.

In [3]:
# =========================================================
# TWO-AGENT GEMINI RAG SYSTEM
# =========================================================

from google import genai
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb

# =========================================================
# AGENT 2: GEMINI (LLM AGENT)
# =========================================================
GEMINI_API_KEY = "--your GEMINI Key--"
MODEL_NAME = "models/gemini-flash-latest"

llm_client = genai.Client(api_key=GEMINI_API_KEY)

class GeminiAgent:
    def answer(self, context, question):
        prompt = f"""
You are a document-grounded AI assistant.

Rules:
- Answer ONLY using the provided context.
- If the answer is not found, say exactly:
  Not found in the document.

Context:
{context}

Question:
{question}
"""
        response = llm_client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt
        )
        return response.text


# =========================================================
# AGENT 1: RETRIEVER AGENT
# =========================================================
class RetrieverAgent:
    def __init__(self, pdf_path):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.client = chromadb.Client()
        self.collection = self.client.get_or_create_collection("two-agent-docs")

        self._ingest_pdf(pdf_path)

    def _ingest_pdf(self, pdf_path):
        reader = PdfReader(pdf_path)
        text = "\n".join(page.extract_text() for page in reader.pages)

        words = text.split()
        chunks = [
            " ".join(words[i:i+300])
            for i in range(0, len(words), 300)
        ]

        for i, chunk in enumerate(chunks):
            self.collection.add(
                documents=[chunk],
                ids=[str(i)],
                embeddings=[self.embedder.encode(chunk).tolist()]
            )

    def retrieve(self, question, k=3):
        q_emb = self.embedder.encode(question).tolist()
        results = self.collection.query(
            query_embeddings=[q_emb],
            n_results=k
        )
        return "\n".join(results["documents"][0])


# =========================================================
# ORCHESTRATOR (AGENT COORDINATOR)
# =========================================================
retriever_agent = RetrieverAgent("Ai_Test_Book.pdf")
llm_agent = GeminiAgent()

# =========================================================
# LIMIT TO 3 QUESTIONS
# =========================================================
MAX_QUESTIONS = 3
question_count = 0

questions = [
    "What is Artificial Intelligence?",
    "What is your salary?",
    "Explain agent-based AI systems."
]

# =========================================================
# RUN TWO-AGENT RAG
# =========================================================
for question in questions:
    if question_count >= MAX_QUESTIONS:
        print("❌ Question limit reached. Session ended.")
        break

    print("\n--------------------------------")
    print(f"Question: {question}")

    # AGENT 1 → RETRIEVE
    context = retriever_agent.retrieve(question)

    # AGENT 2 → REASON
    answer = llm_agent.answer(context, question)

    print(f"Answer: {answer}")

    question_count += 1




--------------------------------
Question: What is Artificial Intelligence?
Answer: Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding.

--------------------------------
Question: What is your salary?
Answer: Not found in the document.

--------------------------------
Question: Explain agent-based AI systems.
Answer: Agent-based systems consist of autonomous components called agents. Each agent has a goal, memory, and decision logic. Multi-agent systems improve reliability and scalability.
